In [1]:
### 1: Get mouse coordiantes for hg ee and neg
#awk '{print $1}' ee_dna_seqs.tsv |grep -Ff - lastznet_hg38_all_exons_treated.bed |sed 's/^chr//' > hg38_ee_mm39_coords.bed
#awk '{print $1}' ctrlNeg_dna_seqs.tsv |grep -Ff - lastznet_hg38_all_exons_treated.bed |sed 's/^chr//' > hg38_neg_mm39_coords.bed

### 2: Get fasta sequences of mouse for those coords
#bedtools getfasta -name+ -fi Mus_musculus.GRCm39.dna.primary_assembly.fa -bed hg38_ee_mm39_coords.bed -fo hg38_ee_mm39_seq.fasta
#bedtools getfasta -name+ -fi Mus_musculus.GRCm39.dna.primary_assembly.fa -bed hg38_neg_mm39_coords.bed -fo hg38_neg_mm39_seq.fasta

In [1]:
### Load data
def reverse_complement(seq):
    dna_seq = seq.upper()
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C', 'N': 'N'}
    return ''.join(complement[base] for base in reversed(dna_seq))

ee_hg_seq_dna={}
with open("/home/mouren/Data/revisions/dn_ds/ee_dna_seqs.tsv") as file:
    for line in file:            
        if line.strip().split()[0][-1] == "f":
            ee_hg_seq_dna[line.strip().split()[0]] = line.strip().split()[1]
        else:
            ee_hg_seq_dna[line.strip().split()[0]] = reverse_complement(line.strip().split()[1])

neg_hg_seq_dna={}
with open("/home/mouren/Data/revisions/dn_ds/ctrlNeg_dna_seqs.tsv") as file:
    for line in file:            
        if line.strip().split()[0][-1] == "f":
            neg_hg_seq_dna[line.strip().split()[0]] = line.strip().split()[1]
        else:
            neg_hg_seq_dna[line.strip().split()[0]] = reverse_complement(line.strip().split()[1])

ee_hg_seq_aa={}
with open("/home/mouren/Data/revisions/dn_ds/ee_aa_seqs_hg38.txt") as file:
    for line in file:            
        ee_hg_seq_aa[line.strip().split()[0]] = line.strip().split()[1]

neg_hg_seq_aa={}
with open("/home/mouren/Data/revisions/dn_ds/neg_aa_seqs_hg38.txt") as file:
    for line in file:       
        try:    
            if line.strip().split()[1] != "false":
                neg_hg_seq_aa[line.strip().split()[0]] = line.strip().split()[1]
        except IndexError:
            continue

## Corresponding seqs in mouse 
ee_mm_seq_dna={}
with open("/home/mouren/Data/revisions/dn_ds/hg38_ee_mm39_seq.fasta") as file:
    id = ""
    for line in file:         
        if line[0] == ">":
            id = (line.strip().split(":")[0]).split(">")[1]
            continue
        else:
            if id[-1] == "f":
                ee_mm_seq_dna[id] = line.strip().split()
            else:
                ee_mm_seq_dna[id] = reverse_complement(line.strip().split()[0])

neg_mm_seq_dna={}
with open("/home/mouren/Data/revisions/dn_ds/hg38_neg_mm39_seq.fasta") as file:
    id = ""
    for line in file:         
        if line[0] == ">":
            id = (line.strip().split(":")[0]).split(">")[1]
            continue
        else:
            if id[-1] == "f":
                neg_mm_seq_dna[id] = line.strip().split()
            else:
                neg_mm_seq_dna[id] = reverse_complement(line.strip().split()[0])

In [2]:
### Trim dna sequences so that that 
from Bio.Seq import Seq

def map_aa_to_dna(dna_seq, aa_seq):
    dna_seq = dna_seq.upper()
    aa_seq = aa_seq.upper()
    
    for offset in range(3):
        # translate from the current offset
        translated_seq = str(Seq(dna_seq[offset:]).translate(to_stop=False))
        
        # Case 1: Exact match starting from current offset
        if translated_seq.startswith(aa_seq):
            return offset, aa_seq, translated_seq
        
        # Case 2: Skip first AA because incomplete codon (missing bases)
        if translated_seq.startswith(aa_seq[1:]):
            return offset, aa_seq[1:], translated_seq
        
        # Case 3: translated_seq missing first AA, check starting from second AA
        if translated_seq[1:].startswith(aa_seq[1:]):
            return offset + 3, aa_seq[1:], translated_seq[1:]
        
    return None, None, None

ee_hg_seq_dna_trimmed = {}
ee_mm_seq_dna_trimmed = {}
for exon_id, dna_seq in ee_hg_seq_dna.items():
    #check if exon was lifted successfully 
    if exon_id in ee_mm_seq_dna:
        mm_seq = ee_mm_seq_dna[exon_id]
    else:
        continue

    #check if we have aa seq for trimming
    if exon_id in ee_hg_seq_aa:
        aa_seq = ee_hg_seq_aa[exon_id]
    else:
        continue

    offset, matched_aa, translated_dna = map_aa_to_dna(dna_seq, aa_seq)
    trimmed_dna_ee_hg = dna_seq[offset:]
    trimmed_dna_ee_mm = mm_seq[offset:]

    trimmed_dna_ee_hg_str = ''.join(base for base in trimmed_dna_ee_hg)
    trimmed_dna_ee_mm_str = ''.join(base for base in trimmed_dna_ee_mm)
    

    if trimmed_dna_ee_hg_str != "" and trimmed_dna_ee_mm_str != "" and len(trimmed_dna_ee_hg_str) == len(trimmed_dna_ee_mm_str):
        ee_hg_seq_dna_trimmed[exon_id] = trimmed_dna_ee_hg_str
        ee_mm_seq_dna_trimmed[exon_id] = trimmed_dna_ee_mm_str

neg_hg_seq_dna_trimmed = {}
neg_mm_seq_dna_trimmed = {}
for exon_id, dna_seq in neg_hg_seq_dna.items():
    #check if exon was lifted successfully 
    if exon_id in neg_mm_seq_dna:
        mm_seq = neg_mm_seq_dna[exon_id]
    else:
        continue

    #check if we have aa seq for trimming
    if exon_id in neg_hg_seq_aa:
        aa_seq = neg_hg_seq_aa[exon_id]
    else:
        continue

    offset, matched_aa, translated_dna = map_aa_to_dna(dna_seq, aa_seq)
    trimmed_dna_neg_hg = dna_seq[offset:]
    trimmed_dna_neg_mm = mm_seq[offset:]

    trimmed_dna_neg_hg_str = ''.join(base for base in trimmed_dna_neg_hg)
    trimmed_dna_neg_mm_str = ''.join(base for base in trimmed_dna_neg_mm)
    

    if trimmed_dna_neg_hg_str != "" and trimmed_dna_neg_mm_str != "" and len(trimmed_dna_neg_hg_str) == len(trimmed_dna_neg_mm_str):
        neg_hg_seq_dna_trimmed[exon_id] = trimmed_dna_neg_hg_str
        neg_mm_seq_dna_trimmed[exon_id] = trimmed_dna_neg_mm_str

/home/mouren/miniconda3/lib/python3.9/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [3]:
#### Code to calculate dnds
import copy
from aa import codons
import math
import sys

#split the sequence
def split_seq(seq,n):
    s=[]
    for i in range(0,len(seq),n):
        s.append(seq[i:i+n])
    return s

#Convert codon to amino acid
def trans_2_condon(seq):
    codons_list=[]
    for base in split_seq(seq,3):
        try:
            codons_list.append(codons[base])
        except KeyError:
            continue
    return codons_list

#find the difference of 2 codon
def diff(codon1,codon2):
    df=[]
    for base in range(0,3):
        if codon1[base]!=codon2[base]:
            df.append(base)
    return df

#find the possible pathway of amino acid conversion
def trans(l):
    if len(l)==1:
        return [l]
    if len(l)==2:
        return [
            l,
            [l[1],l[0]]
        ]
    if len(l)==3:
        return [l,
[l[0],l[2],l[1]],
[l[1],l[0],l[2]],
[l[1],l[2],l[0]],
[l[2],l[0],l[1]],
[l[2],l[1],l[0]]
]

#find the pathway of of amino acid conversion
def sd_nd(codon1,codon2):
    snp_pos=diff(codon1,codon2)
    pathway=trans(snp_pos)
    combine=''
    for aa in pathway:
        new_condon=copy.deepcopy(codon1)
        combine=combine+''.join(new_condon)
        for base in aa:
            new_condon[base]=codon2[base]
            combine=combine+''.join(new_condon)
    return (combine)


#return the number of non-synonymous
def count_diff(aa_list):
    count=0
    for i in range(0,len(aa_list)-1):
        if aa_list[i]!=aa_list[i+1]:
            count+=1
    return count

#Input the out put of function sd_nd(), return sd and nd
def sd_nd_cal(combine):
    nd=0
    pep=trans_2_condon(combine)
    if len(pep)==2:
        if pep[0]==pep[1]:
            return [1,0]
        else:
            return [0,1]
    elif len(pep)==6:
        peplist=split_seq(pep,2)
        for i in peplist:
            nd+=count_diff(i)
        return [(4-nd)/2,nd/2]
    else:
        peplist=split_seq(pep,4)
        for i in peplist:
            nd+=count_diff(i)
            #print (i,count_diff(i))
        return [(18-nd)/6,nd/6]

def test_syno(codon1,codon2):
    try:
        if codons[codon1]==codons[codon2]:
            return 1
        else:
            return 0
    except KeyError:
        return 0

#Compute the number of synonymous sites (s)
def S_n(codon):
    base={'A','G','C','T'}
    s_num=0
    for b in [0,1,2]:
        try:
            other_base=base-{codon[b]}
        except KeyError:
            continue
        for newbase in other_base:
            try:
                new_condon=codon[:b] + newbase + codon[b + 1:]
            except KeyError:
                continue
            s_num+=test_syno(codon,new_condon)
    return (s_num)

#Calculate the number of synonymous sites (s) and the number of nonsynonymous sites (n) for a sequence
def s_n_seq(seq):
    condon_list=split_seq(seq,3)
    sd=0
    for i in condon_list:
        sd+=S_n(i)
    nd=len(condon_list)*3-sd/3
    return [sd/3,nd]

#Compute the number of synonymous and nonsynonymous nucleotide differences between a pair of homologous sequences
def ds_dn_seq(seq1,seq2):
    condon_list1=split_seq(seq1,3)
    condon_list2=split_seq(seq2,3)
    sd_nd_val=[0,0]
    for cond in range(0,len(condon_list1)):
        c1=condon_list1[cond]
        c2=condon_list2[cond]
        if c1==c2:
            pass
        else:
            xy=sd_nd(list(c1),list(c2))
            sd_nd_exp2=sd_nd_cal(xy)
            sd_nd_val[0]+=sd_nd_exp2[0]
            sd_nd_val[1]+=sd_nd_exp2[1]
    return sd_nd_val

#Compute the dn/ds and pn/ps value
def dnds(seq1,seq2):
    assert len(seq1)==len(seq2),'Fatal err:length(seq1)!=length(seq2)'
    if len(seq1)%3!=0: #'Fatal err:The length of input seq is not divisible by 3'
        seq1 = seq1 + "N"
        seq2 = seq2 + "N"
        if len(seq1)%3!=0:
            seq1 = seq1 + "N"
            seq2 = seq2 + "N"
    assert len(seq1)%3==0,'Fatal err:The length of input seq is not divisible by 3'
    if seq1==seq2:
        return None,None,None,None,None
    sdnd1=s_n_seq(seq1)
    sdnd2=s_n_seq(seq2)
    sdnd=[(sdnd1[0]+sdnd2[0])/2,(sdnd1[1]+sdnd2[1])/2]
    sd=ds_dn_seq(seq1,seq2)
    ps=sd[0]/sdnd[0]
    pn=sd[1]/sdnd[1]
    
    if ps >= 0.75 or pn >= 0.75:
        #print("Too divergent for Jukes-Cantor correction. Skipping.")
        return pn, ps, float('nan'), float('nan'), float('nan')
    
    try:
        ds=-0.75*math.log(1-(4*ps)/3)
    except ValueError:
        print(ps,pn,sdnd1,sdnd2,sd)
    dn=-0.75*math.log(1-(4*pn)/3)

    if ds == 0:
        dnds = float('inf') if dn > 0 else float('nan')  # or set dnds = None
    else:
        dnds = dn / ds
    return pn,ps,dn,ds,dnds

In [5]:
### Compute dN/dS
import pandas as pd 

total_dnds_ee = []
results_ee = [["ID","pN","pS","dN","dS","dNdS"]]
sum_dn_ee = 0
sum_ds_ee = 0
for key,seq1 in ee_hg_seq_dna_trimmed.items():
    seq2 = ee_mm_seq_dna_trimmed[key]
    pn,ps,dn,ds,dn_ds = dnds(seq1,seq2)
    if ds is not None and ds > 0 and dn is not None:
        total_dnds_ee.append(dn / ds)
        sum_dn_ee += dn
        sum_ds_ee += ds
    results_ee.append([key,pn,ps,dn,ds,dn_ds])

total_dnds_neg = []
results_neg = [["ID","pN","pS","dN","dS","dNdS"]]
sum_dn_neg = 0
sum_ds_neg = 0
for key,seq1 in neg_hg_seq_dna_trimmed.items():
    seq2 = neg_mm_seq_dna_trimmed[key]
    pn,ps,dn,ds,dn_ds = dnds(seq1,seq2)
    if ds is not None and ds > 0 and dn is not None:
        total_dnds_neg.append(dn / ds)
        sum_dn_neg += dn
        sum_ds_neg += ds
    results_neg.append([key,pn,ps,dn,ds,dn_ds])

# Compute global dN/dS
global_dnds_ee = sum_dn_ee / sum_ds_ee if sum_ds_ee > 0 else None
global_dnds_neg = sum_dn_neg / sum_ds_neg if sum_ds_neg > 0 else None

# Append total row
results_ee.append(["TOTAL", "NA", "NA", sum_dn_ee, sum_ds_ee, global_dnds_ee])
results_neg.append(["TOTAL", "NA", "NA", sum_dn_neg, sum_ds_neg, global_dnds_neg])

df_ee = pd.DataFrame(results_ee).astype(str)
df_ee.to_csv("//home/mouren/Data/revisions/dn_ds/hg_ee_dnds_mm.tsv",sep="\t",header=False,index=False)

df_neg = pd.DataFrame(results_neg).astype(str)
df_neg.to_csv("//home/mouren/Data/revisions/dn_ds/hg_neg_dnds_mm.tsv",sep="\t",header=False,index=False)

In [17]:
### Perform statistical test on per-exon dN/dS
from scipy.stats import chi2_contingency,mannwhitneyu
import numpy as np

# Convert to float, remove invalids
def clean_numeric(data):
    cleaned = []
    for v in data:
        try:
            f = float(v)
            if not np.isnan(f):
            #if np.isfinite(f):  # excludes NaN, inf, -inf
                cleaned.append(f)
        except (ValueError, TypeError):
            continue
    return cleaned

clean_ee = clean_numeric(total_dnds_ee)
clean_neg = clean_numeric(total_dnds_neg)

print("median",np.median(clean_ee))
print("median",np.median(clean_neg))

print("mean",np.mean(clean_ee))
print("mean",np.mean(clean_neg))

stat, p_value = mannwhitneyu(clean_ee, clean_neg, alternative='two-sided')

print(f"\nMann-Whitney U Test:")
print(f"U statistic = {stat}")
print(f"p-value = {p_value}")

if p_value < 0.05:
    print("→ Significant difference in medians (p < 0.05)")
else:
    print("→ No significant difference in medians (p ≥ 0.05)")

median 0.08584161580822748
median 0.08821830141414796
mean 0.2687961604907738
mean 0.2335094359605969

Mann-Whitney U Test:
U statistic = 10640535.5
p-value = 0.0006280121415051488
→ Significant difference in medians (p < 0.05)


In [13]:
### TOTAL STATS
# Build 2x2 table
table = np.array([
    [sum_dn_ee, sum_ds_ee],        # Disease: [pN, pS]
    [sum_dn_neg, sum_ds_neg]  # Control: [pN, pS]
])

chi2, p, dof, expected = chi2_contingency(table)

print(f"Chi-squared: {chi2:.4f}")
print(f"p-value: {p:.4g}")
print("Expected counts:\n", expected)

Chi-squared: 53.8741
p-value: 2.138e-13
Expected counts:
 [[1703.04926372 4557.72778686]
 [1339.06369192 3583.62375478]]


In [7]:
### test trimming 
print(ee_hg_seq_dna["ENST00000353662.4_cds_3_0_chr1_1300509_r"])
print(ee_hg_seq_aa["ENST00000353662.4_cds_3_0_chr1_1300509_r"])
print(ee_hg_seq_dna_trimmed["ENST00000353662.4_cds_3_0_chr1_1300509_r"])

offset, matched_aa, translated_dna = map_aa_to_dna("GGATGTGCGGAAGTTCAAGGAGACAAAGAAGCAGTTTGACAAGGTGCGGGAGGACCTGGAGCTGTCCCTGGTGAGGAACGCCCAGGCCCCGAGGCACCGGCCCCACGAGGTGGAGGAAGCCACCGGGGCCCTCACCCTCACCAGGAAGTGCTTCCGCCACCTGGCACTGGACTATGTGCTCCAG", "EDVRKFKETKKQFDKVREDLELSLVRNAQAPRHRPHEVEEATGALTLTRKCFRHLALDYVLQ")
print(offset, "\n", matched_aa, "\n", translated_dna)

print(ee_hg_seq_dna["ENST00000315480.9_cds_8_0_chr9_129863186_f"])
print(ee_hg_seq_aa["ENST00000315480.9_cds_8_0_chr9_129863186_f"])
print(ee_hg_seq_dna_trimmed["ENST00000315480.9_cds_8_0_chr9_129863186_f"])

offset, matched_aa, translated_dna = map_aa_to_dna("CCCGCCGCTGACTCAGTTCTTCTTGGAGTGTGGCGGCCTGGTGCGCACAGATAAGAAGCCAGCCCTGTGCAAGAGCTACCAGAAGCTGGTCTCTGAGGTCTGGCATAAGAAACG", "CPPLTQFFLECGGLVRTDKKPALCKSYQKLVSEVWHKK")
print(offset, "\n", matched_aa, "\n", translated_dna)

print(ee_hg_seq_dna_trimmed["ENST00000353662.4_cds_3_0_chr1_1300509_r"])
print(ee_mm_seq_dna_trimmed["ENST00000353662.4_cds_3_0_chr1_1300509_r"])

#test dnds calculus
print(ee_hg_seq_dna_trimmed["ENST00000379290.6_cds_13_0_chr1_1196600_f"])
print(ee_mm_seq_dna_trimmed["ENST00000379290.6_cds_13_0_chr1_1196600_f"])

"""print(ee_hg_seq_dna_trimmed["ENST00000315480.9_cds_8_0_chr9_129863186_f"])
print(ee_mm_seq_dna_trimmed["ENST00000315480.9_cds_8_0_chr9_129863186_f"])
"""


GGATGTGCGGAAGTTCAAGGAGACAAAGAAGCAGTTTGACAAGGTGCGGGAGGACCTGGAGCTGTCCCTGGTGAGGAACGCCCAGGCCCCGAGGCACCGGCCCCACGAGGTGGAGGAAGCCACCGGGGCCCTCACCCTCACCAGGAAGTGCTTCCGCCACCTGGCACTGGACTATGTGCTCCAG
EDVRKFKETKKQFDKVREDLELSLVRNAQAPRHRPHEVEEATGALTLTRKCFRHLALDYVLQ
GATGTGCGGAAGTTCAAGGAGACAAAGAAGCAGTTTGACAAGGTGCGGGAGGACCTGGAGCTGTCCCTGGTGAGGAACGCCCAGGCCCCGAGGCACCGGCCCCACGAGGTGGAGGAAGCCACCGGGGCCCTCACCCTCACCAGGAAGTGCTTCCGCCACCTGGCACTGGACTATGTGCTCCAG
1 
 DVRKFKETKKQFDKVREDLELSLVRNAQAPRHRPHEVEEATGALTLTRKCFRHLALDYVLQ 
 DVRKFKETKKQFDKVREDLELSLVRNAQAPRHRPHEVEEATGALTLTRKCFRHLALDYVLQ
CCCGCCGCTGACTCAGTTCTTCTTGGAGTGTGGCGGCCTGGTGCGCACAGATAAGAAGCCAGCCCTGTGCAAGAGCTACCAGAAGCTGGTCTCTGAGGTCTGGCATAAGAAACG
CPPLTQFFLECGGLVRTDKKPALCKSYQKLVSEVWHKK


/home/mouren/miniconda3/lib/python3.9/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


KeyError: 'ENST00000315480.9_cds_8_0_chr9_129863186_f'